<a href="https://colab.research.google.com/github/Prakrut/Fraud_Detection/blob/master/Code/CreditCardFraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries 

In [15]:
# Python
import pandas as pd
import sys, os
import numpy as np 



# Third-Party
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Custom

# Options
pd.set_option('max_columns',500)
pd.set_option('max_rows',50)


### Data 

In [4]:
path = '../Data'
for i in os.listdir(path):
    print (i)

sample_submission.csv
test_identity.csv
test_transaction.csv
train_identity.csv
train_transaction.csv


In [5]:
train_identity = pd.read_csv(path + '/train_identity.csv')
train_transaction = pd.read_csv(path + '/train_transaction.csv')

#### Check for Mutually Exclusive? 

In [6]:
len(set(train_identity.TransactionID.unique()) - set(train_transaction.TransactionID.unique()))


0

In [7]:
len(set(train_transaction.TransactionID.unique()) - set(train_identity.TransactionID.unique()))


446307

#### meaning do a left join on transaction. 

In [8]:
train = train_transaction.merge(train_identity,
                                on = 'TransactionID',
                                how = 'left'
                               )

In [9]:
test = pd.merge(pd.read_csv(path + '/test_transaction.csv'),
                pd.read_csv(path + '/test_identity.csv'),
                on = 'TransactionID',
                how = 'left'
               )

In [10]:
test.shape, train.shape

((506691, 433), (590540, 434))

In [11]:
set(train.columns) - set(test.columns)

{'id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_25',
 'id_26',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_32',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'isFraud'}

In [12]:
set(test.columns) - set(train.columns)


{'id-01',
 'id-02',
 'id-03',
 'id-04',
 'id-05',
 'id-06',
 'id-07',
 'id-08',
 'id-09',
 'id-10',
 'id-11',
 'id-12',
 'id-13',
 'id-14',
 'id-15',
 'id-16',
 'id-17',
 'id-18',
 'id-19',
 'id-20',
 'id-21',
 'id-22',
 'id-23',
 'id-24',
 'id-25',
 'id-26',
 'id-27',
 'id-28',
 'id-29',
 'id-30',
 'id-31',
 'id-32',
 'id-33',
 'id-34',
 'id-35',
 'id-36',
 'id-37',
 'id-38'}

#### Make adjustments and check 

In [13]:
test.columns = test.columns.map(lambda x : x.replace('-','_'))


In [14]:
set(train.columns) - set(test.columns)


{'isFraud'}